In [ ]:
from selenium import webdriver
import numpy as np

import time
from selenium import webdriver

from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located

from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import sys

In [ ]:
import re
from selenium.webdriver import Keys, ActionChains

from selenium.common.exceptions import ElementClickInterceptedException

import pyautogui
import warnings
warnings.filterwarnings("ignore")



options = webdriver.ChromeOptions()

prefs = {"profile.default_content_setting_values.notifications" : 2}

options.add_experimental_option("prefs",prefs)
options.page_load_strategy = 'normal'
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.maximize_window()
driver.get("https://www.citilink.ru/catalog/kompyutery/?ref=mainmenu")

data_link = pd.DataFrame(columns=['link'])

link_new = dict()
link_new.clear()
start_page = 1
start_card = 1


for num in range (start_page, 12):
    print("num = ", num)
    for i in range(start_card, 49):
        try:
            WebDriverWait(driver, 10).until(expected_conditions.visibility_of_element_located((By.XPATH, f'/html/body/div[2]/div/main/section/div[2]/div/div/section/div[2]/div[2]/div[{i}]/div/div[3]/div[1]/a')))
            search_box = driver.find_element(By.XPATH, f'/html/body/div[2]/div/main/section/div[2]/div/div/section/div[2]/div[2]/div[{i}]/div/div[3]/div[1]/a')
            #print("1")                                  /html/body/div[2]/div/main/section/div[2]/div/div/section/div[2]/div[2]/div[4  ]/div/div[3]/div[1]/a
            link_new.update({'link': search_box.get_attribute("href")})                         
            data_link = data_link.append(link_new, ignore_index=True) 
            data_link.to_csv(f'link.csv', index=False)
            #print("2")
        except Exception as e:
            pass

        link_new.clear()
    driver.get(f"https://www.citilink.ru/catalog/kompyutery/?p={num+1}")

    start_card = 1


In [ ]:
import re
from selenium.webdriver import Keys, ActionChains
from selenium.common.exceptions import ElementClickInterceptedException
import pyautogui
import warnings
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

warnings.filterwarnings("ignore")

options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)
options.page_load_strategy = 'eager'
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

#DesiredCapabilities.CHROME = None

driver.maximize_window()
driver.get("https://www.citilink.ru/catalog/kompyutery/?ref=mainmenu")


time.sleep(5)

data_link = pd.read_csv('link.csv')


data = dict()
data_full = pd.read_csv('data.csv')

start_link = data_link['link'][0]
start_index = data_link[data_link['link'] == start_link].index[0]
i = start_index

driver.stop_client()

for link_from_data in data_link.loc[start_index:, 'link']:
    #print("1")
    driver.get(link_from_data)

    #print("2")
    time.sleep(2)
    rating = ''
    num_rating = ''

    try:
        rating = driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div[1]/div[2]/div/div[3]/div/div[1]').text
        data.update({'Рейтинг': rating})
    except Exception:
        pass
    try:
        num_rating = driver.find_element(By.XPATH, f'/html/body/div[2]/div/main/div[1]/div[2]/div/div[3]/div/div[2]/div').text
        num_rating = num_rating.replace(' Отзывa', '')
        num_rating = num_rating.split(' ')[0]
    except Exception:
        pass
    data.update({'Количество отзывов': num_rating})

    #print(rating, num_rating)
    
    try:
        name = driver.find_element(By.XPATH, f'/html/body/div[2]/div/div[5]/div/div/div/div[1]/div/div[2]/div[1]/span').text
    except Exception:
        pass
    try:
        name = driver.find_element(By.XPATH, f'/html/body/div[2]/div/main/div[1]/div[2]/div/div[2]/h1').text
    except Exception:
        pass
            

    try:
        price = driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div[1]/div[2]/div/div[4]/div/div[3]/div/div[1]/div/div/span/span/span[1]').text
    except Exception:
        pass
    try:
        price = driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div[1]/div[2]/div/div[4]/div/div[3]/div/div[2]/div/div[2]/span/span/span[1]').text
    except Exception:                          
        pass
    try:
        price = driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div[1]/div[2]/div/div[4]/div/div[3]/div/div[2]/div/div/span/span/span[1]').text
    except Exception:                          
        pass

    try:
        price = driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div[1]/div[2]/div/div[4]/div/div[3]/div/div[1]/div/div[1]/span').text
    except Exception:                          
        pass

    #print(name, price)
    data.update({"Название": name, "Цена": price})
    data.update({"link": link_from_data})
    if price == 'Нет в наличии':
        WebDriverWait(driver, 10).until(expected_conditions.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div/main/div[1]/div[2]/div/div[4]/div/div[2]/div/div[1]/div[2]/button/span/span[1]')))
        driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div[1]/div[2]/div/div[4]/div/div[2]/div/div[1]/div[2]/button/span/span[1]').click()
    else:
        WebDriverWait(driver, 10).until(expected_conditions.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div/main/div[1]/div[3]/div[1]/div/div/div/div/a[2]/div')))
        driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div[1]/div[3]/div[1]/div/div/div/div/a[2]/div').click()

    
    try :
        WebDriverWait(driver, 3).until(expected_conditions.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div/main/div[1]/div[3]/div[2]/div/div/div[1]/ul/li[2]/div[2]/div/span/span')))
        param = driver.find_element(By.XPATH, f'/html/body/div[2]/div/main/div[1]/div[3]/div[2]/div/div/div[1]/ul/li[1]/div/div/span/span').text
        value = driver.find_element(By.XPATH, f'/html/body/div[2]/div/main/div[1]/div[3]/div[2]/div/div/div[1]/ul/li[1]/div/span/span').text
        data.update({param: value})
    except Exception:
        pass
                                   
    for j in range(2,20):
        for k in range(1, 15):
            try:
                param = driver.find_element(By.XPATH, f'/html/body/div[2]/div/main/div[1]/div[3]/div[2]/div/div/div[1]/ul/li[{j}]/div[{k}]/div/span/span').text
                value = driver.find_element(By.XPATH, f'/html/body/div[2]/div/main/div[1]/div[3]/div[2]/div/div/div[1]/ul/li[{j}]/div[{k}]/span/span').text
                                                       #/html/body/div[2]/div/main/div[1]/div[4]/div[2]/div/div/div[1]/ul/li[1]/div[1]/div/span/span
                data.update({param: value})
            except Exception:
                pass
                #print("Элемент не найден")

    data_full = data_full.append(data, ignore_index=True) 
    i += 1
    data_full.to_csv(f'data.csv', index=False)
    data.clear()
    print(f"Card: {i} FINISHED !")